In [1]:
import pandas as pd 
import torch 
import torch.nn as nn 
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import numpy as np 
import argparse
from copy import deepcopy #Add Deepcopy for args
import visdom
vis = visdom.Visdom()

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

print(torch.__version__)
%matplotlib inline
%pylab inline
pylab.rcParams['figure.figsize'] = (15, 9)

Setting up a new session...


1.4.0
Populating the interactive namespace from numpy and matplotlib


# 1. Data loading & Preprocessing

### 1.1 Data loading

In [2]:
cd /Users/jinsungpark/Desktop/Data_river/data04

[Errno 2] No such file or directory: '/Users/jinsungpark/Desktop/Data_river/data04'
/Users/jinsungpark/Desktop/jupyter


In [3]:
pwd #현재 경로 확인

'/Users/jinsungpark/Desktop/jupyter'

In [4]:
ls #현재경로에 있는 항목 확인

Correlation_analysis.ipynb
Data_Preprocessing.ipynb
Data_river/
Lab10_Stock_Price_Prediction_with_LSTM.ipynb
Pandas_tutorial.ipynb
Project_Git/
Project_River_Backup.ipynb
Project_River_Backup_change_TN.ipynb
Project_River_Backup_change_TP.ipynb
Project_River_Backup_change_test.ver.ipynb
Project_River_Find_BestModel.ipynb
Project_River_Find_BestModel_01.ipynb
Project_River_matplotlib.ipynb
Project_River_matplotlib_01.ipynb
Untitled.ipynb
results/


In [5]:
cd /Users/jinsungpark/Desktop/jupyter/Data_river/data04

/Users/jinsungpark/Desktop/jupyter/Data_river/data04


In [6]:
UpStream_data = pd.read_excel('DS_Data_edit_log.xlsx')
DownStream_data = pd.read_excel('NG_Data_edit_log.xlsx')

In [7]:
print(UpStream_data.columns)
# print(DownStream_data.columns)

Index(['Date', 'DS_DO', 'DS_BOD', 'DS_COD', 'DS_SS', 'DS_TN', 'DS_TP',
       'DS_Chl_a', 'DS_Cells', 'GJ_Deep', 'GJ_Level', 'GJ_Outflow',
       'DaeGu_Rain', 'DaeGu_Solar', 'SeoBu_COD', 'SeoBu_SS', 'SeoBu_TN',
       'SeoBu_TP', 'SeoBu_Flow_mean', 'SeoBu_Flow_day', 'SeoBu_COD_load',
       'SeoBu_SS_load', 'SeoBu_TN_load', 'SeoBu_TP_load', 'SungSeo_COD',
       'SungSeo_SS', 'SungSeo_TN', 'SungSeo_TP', 'SungSeo_Flow_mean',
       'SungSeo_Flow_day', 'SungSeo_COD_load', 'SungSeo_SS_load',
       'SungSeo_TN_load', 'SungSeo_TP_load', 'GumHo_DO', 'GumHo_BOD',
       'GumHo_COD', 'GumHo_SS', 'GumHo_TN', 'GumHo_TP', 'GumHo_Chl_a',
       'GumHo_Flow', 'DS_Temp', 'GumHo_Temp'],
      dtype='object')


In [8]:
#날짜 인덱스화
UpData = UpStream_data.set_index('Date')
DownData = DownStream_data.set_index('Date')

In [9]:
UpData.info()

<class 'pandas.core.frame.DataFrame'>
Index: 349 entries, 2013-01-07 to 2019-09-30
Data columns (total 43 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   DS_DO              349 non-null    float64
 1   DS_BOD             349 non-null    float64
 2   DS_COD             349 non-null    float64
 3   DS_SS              349 non-null    float64
 4   DS_TN              349 non-null    float64
 5   DS_TP              349 non-null    float64
 6   DS_Chl_a           349 non-null    float64
 7   DS_Cells           349 non-null    float64
 8   GJ_Deep            349 non-null    float64
 9   GJ_Level           349 non-null    float64
 10  GJ_Outflow         349 non-null    float64
 11  DaeGu_Rain         349 non-null    float64
 12  DaeGu_Solar        349 non-null    float64
 13  SeoBu_COD          349 non-null    float64
 14  SeoBu_SS           349 non-null    float64
 15  SeoBu_TN           349 non-null    float64
 16  SeoBu_TP       

In [10]:
#넣고싶은 상류 항목 컬럼 선택
UpData = UpData.iloc[:,[0,2,3,5,10,11,28,29,33,36,38,40,41,42]]
print(UpData.columns)

Index(['DS_DO', 'DS_COD', 'DS_SS', 'DS_TP', 'GJ_Outflow', 'DaeGu_Rain',
       'SungSeo_Flow_day', 'SungSeo_COD_load', 'GumHo_DO', 'GumHo_SS',
       'GumHo_TP', 'GumHo_Flow', 'DS_Temp', 'GumHo_Temp'],
      dtype='object')


In [11]:
UpData.to_csv('UpData.csv')

In [12]:
DownData.info()

<class 'pandas.core.frame.DataFrame'>
Index: 349 entries, 2013-01-07 to 2019-09-30
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   NG_DO     349 non-null    float64
 1   NG_BOD    349 non-null    float64
 2   NG_COD    349 non-null    float64
 3   NG_SS     349 non-null    float64
 4   NG_TN     349 non-null    float64
 5   NG_TP     349 non-null    float64
 6   NG_Chl_a  349 non-null    float64
dtypes: float64(7)
memory usage: 21.8+ KB


In [13]:
#알고싶은 하류 항목 컬럼 넘버 넣기('Date'항목이 인덱스화 돼서 컬럼 넘버가 -1씩 됨)
Colum = 5
print(DownData.columns[Colum])

NG_TP


### 1.2 Data Preprocessing(normalization)

In [14]:
from sklearn.preprocessing import MinMaxScaler

UpScaler = MinMaxScaler() #상류데이터용
DownScaler = MinMaxScaler() #하류데이터용

#나중에 결과를 DeNormalizing 하기 위해 나누어 사용 하였다.

def DeNormalize(Y, Data_name, column_num, Scaler_Type):
    
    data = Data_name
    Scaler = Scaler_Type
    
    _max = Scaler.data_max_[column_num] # 역정규화 하려는 데이터의 컬럼 번호
    _min = Scaler.data_min_[column_num] 
    
    X = Y*(_max-_min) + _min
    
    return X

In [15]:
#데이터 정규화
UpData = pd.DataFrame(UpScaler.fit_transform(UpData), columns=UpData.columns, index=UpData.index)
DownData = pd.DataFrame(DownScaler.fit_transform(DownData), columns=DownData.columns, index=DownData.index)

print(UpData.isna().sum())

DS_DO               0
DS_COD              0
DS_SS               0
DS_TP               0
GJ_Outflow          0
DaeGu_Rain          0
SungSeo_Flow_day    0
SungSeo_COD_load    0
GumHo_DO            0
GumHo_SS            0
GumHo_TP            0
GumHo_Flow          0
DS_Temp             0
GumHo_Temp          0
dtype: int64


#2. Data Preparation

In [16]:
class RiverDataset(Dataset):
    def __init__(self, UpData, DownData, x_frames, y_frames, start, end):
        
        self.x_frames = x_frames
        self.y_frames = y_frames
        
        self.start = start
        self.end = end

        self.UpData = UpData[start:end]
        self.DownData = DownData[start:end]

    def __len__(self):
        return len(self.UpData) - (self.x_frames + self.y_frames) + 1
    #데이터를 전처리 할때 UpData와 DownData의 길이가 동일해짐(날짜를 동일한것만 추출해야 하므로), 따라서 전체길이는 둘중 하나를 사용

    def __getitem__(self, idx):
        idx += self.x_frames

        X = self.UpData.iloc[idx-self.x_frames:idx].values
        Y = self.DownData.iloc[idx:idx+self.y_frames].values
        
        return X, Y

# 3. Model Define

In [17]:
class LSTM(nn.Module):
    
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, batch_size, dropout, use_bn):
        super(LSTM, self).__init__()
        self.input_dim = input_dim 
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.num_layers = num_layers

        self.batch_size = batch_size
        self.dropout = dropout
        self.use_bn = use_bn 
        
        self.lstm = nn.LSTM(self.input_dim, self.hidden_dim, self.num_layers) #
        self.hidden = self.init_hidden()
        self.regressor = self.make_regressor()
        
    def init_hidden(self):
        return (torch.zeros(self.num_layers, self.batch_size, self.hidden_dim),
                torch.zeros(self.num_layers, self.batch_size, self.hidden_dim))
    
    def make_regressor(self):
        layers = []
        if self.use_bn:
            layers.append(nn.BatchNorm1d(self.hidden_dim))
        layers.append(nn.Dropout(self.dropout))
        
        layers.append(nn.Linear(self.hidden_dim, self.hidden_dim // 2))
        layers.append(nn.ReLU())
        layers.append(nn.Linear(self.hidden_dim // 2, self.output_dim))
        regressor = nn.Sequential(*layers)
        return regressor
    
    def forward(self, x):
        lstm_out, self.hidden = self.lstm(x, self.hidden)
        y_pred = self.regressor(lstm_out[-1].view(self.batch_size, -1))
        return y_pred

In [18]:
# 정확도 : 예측확률을 100%로 봤을때 MAPE에 따른 오차비율을 빼줌 (100-MAPE) ##RMSE, MAPE 두개로 볼 수 있게
def MAPE(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [19]:
def RMSE(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred, multioutput='raw_values')
    return np.sqrt(mse)

In [20]:
def R2(y_true, y_pred):
    R2_score = r2_score(y_true, y_pred, multioutput='raw_values')
    return R2_score

# 4. Train, Validate, Test

In [21]:
def train(model, partition, optimizer, loss_fn, args):
    trainloader = DataLoader(partition['train'],
                             batch_size=args.batch_size,
                             shuffle=False, drop_last=True)
    model.train()
    model.zero_grad()
    optimizer.zero_grad()

    bat_siz = args.batch_size
    pred = []
    true = []
    pred_results = []
    true_results = []
    train_acc = 0.0
    train_loss = 0.0
    
    for i, (X, y) in enumerate(trainloader):

        X = X.transpose(0, 1).float().to(args.device)#파이토치는 순서가 달라서 바꿔줌
        y_true = y[:, :, Colum].float().to(args.device)
        model.zero_grad()
        optimizer.zero_grad()
        model.hidden = [hidden.to(args.device) for hidden in model.init_hidden()]

        y_pred = model(X)
        loss = loss_fn(y_pred.view(-1), y_true.view(-1))
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

        pred.append(y_pred)
        true.append(y_true)

    # ==== test 데이터 시각화를 위해 x,y데이터 저장 (배치사이즈의 텐서들을 한개씩 추가해주기) ==== #
    for i in range(len(trainloader)):
        tems1 = pred[i].view(bat_siz).cpu().detach().numpy()
        tems2 = true[i].view(bat_siz).cpu().detach().numpy()
        
        for j in range(bat_siz):
            value1 = np.exp(DeNormalize(tems1[j], DownData, Colum, DownScaler))
            value2 = np.exp(DeNormalize(tems2[j], DownData, Colum, DownScaler))
            
            pred_results.append(value1)
            true_results.append(value2)
    # ========================================================================== #   

    train_loss = train_loss / len(trainloader)
    train_acc1 = RMSE(np.array(true_results), np.array(pred_results))
    train_acc2 = R2(np.array(true_results), np.array(pred_results))
#     train_acc3 = (100 - MAPE(np.array(true_results), np.array(pred_results)))

    return model, train_loss, train_acc1[0], train_acc2[0]

In [22]:
def validate(model, partition, loss_fn, args):
    valloader = DataLoader(partition['val'],
                           batch_size=args.batch_size,
                           shuffle=False, drop_last=True)
    model.eval()

    bat_siz = args.batch_size
    pred = []
    true = []
    pred_results = []
    true_results = []
    val_acc = 0.0
    val_loss = 0.0
    
    with torch.no_grad():
        for i, (X, y) in enumerate(valloader):

            X = X.transpose(0, 1).float().to(args.device)
            y_true = y[:, :, Colum].float().to(args.device)
            model.hidden = [hidden.to(args.device) for hidden in model.init_hidden()]

            y_pred = model(X)
            loss = loss_fn(y_pred.view(-1), y_true.view(-1))

            val_loss += loss.item()

            pred.append(y_pred)
            true.append(y_true)

    # ==== test 데이터 시각화를 위해 x,y데이터 저장 (배치사이즈의 텐서들을 한개씩 추가해주기) ==== #
    for i in range(len(valloader)):
        tems1 = pred[i].view(bat_siz).cpu().detach().numpy()
        tems2 = true[i].view(bat_siz).cpu().detach().numpy()
        
        for j in range(bat_siz):
            value1 = np.exp(DeNormalize(tems1[j], DownData, Colum, DownScaler))
            value2 = np.exp(DeNormalize(tems2[j], DownData, Colum, DownScaler))
            pred_results.append(value1)
            true_results.append(value2)
    # ========================================================================== #   
    
    val_loss = val_loss / len(valloader)
    val_acc1 = RMSE(np.array(true_results), np.array(pred_results))
    val_acc2 = R2(np.array(true_results), np.array(pred_results))
#     val_acc3 = (100 - MAPE(np.array(true_results), np.array(pred_results)))

    
    return val_loss, val_acc1[0], val_acc2[0]

In [23]:
def test(model, partition, args):
    testloader = DataLoader(partition['test'],
                           batch_size=args.batch_size,
                           shuffle=False, drop_last=True)
    model.eval()

    bat_siz = args.batch_size
    pred = []
    true = []
    pred_results = []
    true_results = []
    test_acc = 0.0
    
    with torch.no_grad():
        for i, (X, y) in enumerate(testloader):
            X = X.transpose(0, 1).float().to(args.device)
            y_true = y[:, :, Colum].float().to(args.device)
            model.hidden = [hidden.to(args.device) for hidden in model.init_hidden()]

            y_pred = model(X)

            pred.append(y_pred)
            true.append(y_true)

    # ==== test 데이터 시각화를 위해 x,y데이터 저장 ==== #
    for i in range(len(testloader)):
        tems1 = pred[i].view(bat_siz).cpu().detach().numpy()
        tems2 = true[i].view(bat_siz).cpu().detach().numpy()
        
        for j in range(bat_siz):
            value1 = np.exp(DeNormalize(tems1[j], DownData, Colum, DownScaler))
            value2 = np.exp(DeNormalize(tems2[j], DownData, Colum, DownScaler))
            
            pred_results.append(value1)
            true_results.append(value2)
    # ========================================== #   

    test_acc1 =  RMSE(np.array( true_results), np.array(pred_results))
    test_acc2 =  R2(np.array( true_results), np.array(pred_results))
#     test_acc3 =  (100 - MAPE(np.array( true_results), np.array(pred_results)))
    
    return test_acc1[0], test_acc2[0], pred_results, true_results

# 5. Experiment

In [33]:
def experiment(partition, args):

    model = LSTM(args.input_dim, args.hid_dim, args.y_frames, args.n_layers, args.batch_size, args.dropout, args.use_bn)
    model.to(args.device)
#     loss_fn = torch.nn.MSELoss() ##loss는 mse를 사용
#     loss_fn = nn.MSELoss()
    
    if args.loss == 'MSELoss':
        loss_fn = torch.nn.MSELoss()
        loss_fn = nn.MSELoss()
    elif args.loss == 'L1Loss':
        loss_fn = torch.nn.L1Loss()
        loss_fn = nn.L1Loss()
    elif args.loss == 'PoissonNLLLoss':
        loss_fn = torch.nn.PoissonNLLLoss()
        loss_fn = nn.PoissonNLLLoss()
    elif args.loss == 'KLDivLoss':
        loss_fn = torch.nn.KLDivLoss()
        loss_fn = nn.KLDivLoss()
    elif args.loss == 'BCELoss':
        loss_fn = torch.nn.BCELoss()
        loss_fn = nn.BCELoss()
    elif args.loss == 'BCEWithLogitsLoss':
        loss_fn = torch.nn.BCEWithLogitsLoss()
        loss_fn = nn.BCEWithLogitsLoss()
    else:
        raise ValueError('In-valid LossFuction choice')
    
    
    
    if args.optim == 'SGD':
        optimizer = optim.SGD(model.parameters(), lr=args.lr, weight_decay=args.l2)
    elif args.optim == 'RMSprop':
        optimizer = optim.RMSprop(model.parameters(), lr=args.lr, weight_decay=args.l2)
    elif args.optim == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.l2)
    else:
        raise ValueError('In-valid optimizer choice')
    
    # ===== List for epoch-wise data ====== #
    train_losses = []
    val_losses = []
    train_accs_RMSE = []
    train_accs_R2 = []
    val_accs_RMSE = []
    val_accs_R2 = []
    axis = []
    # ===================================== #
    
    ## model starting point ##    
    ts = time.time()
    model, train_loss, train_acc_RMSE, train_acc_R2 = train(model, partition, optimizer, loss_fn, args)
    val_loss, val_acc_RMSE, val_acc_R2 = validate(model, partition, loss_fn, args)
    te = time.time()

    # ====== Add Epoch Data ====== #
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accs_RMSE.append(train_acc_RMSE)
    val_accs_RMSE.append(val_acc_RMSE)
    train_accs_R2.append(train_acc_R2)
    val_accs_R2.append(val_acc_R2)
    # ============================ #
    
    result = {}
    
    # # ===== Visdom visualizing ================================================================================== #
    axis.append(0)
    
    plot1 = vis.line(Y=torch.cat((torch.Tensor(train_losses).view(-1,1), torch.Tensor(val_losses).view(-1,1)), -1),
                     X=torch.cat((torch.Tensor(axis).view(-1,1), torch.Tensor(axis).view(-1,1)), -1),
                     opts=dict(title='exp_{}_loss'.format(num), legend=['train_loss','val_loss'], showlegend=True))
    
    plot2 = vis.line(Y=torch.cat((torch.Tensor(train_accs_RMSE).view(-1,1), torch.Tensor(val_accs_RMSE).view(-1,1)), -1),
                     X=torch.cat((torch.Tensor(axis).view(-1,1), torch.Tensor(axis).view(-1,1)), -1),
                     opts=dict(title='exp_{}_acc_RMSE'.format(num), legend=['train_acc','val_acc'], showlegend=True))
    # # =========================================================================================================== #
    
    print('Epoch {}, Acc_RMSE(train/val): {:2.2f}/{:2.2f}, Loss(train/val) {:2.5f}/{:2.5f}. Took {:2.2f} sec'
          .format(0, train_acc_RMSE, val_acc_RMSE, train_loss, val_loss, te-ts))
    
    for epoch in range(args.epoch-1):  # loop over the dataset multiple times
        
        ts = time.time()
        model, train_loss, train_acc_RMSE, train_acc_R2 = train(model, partition, optimizer, loss_fn, args)
        val_loss, val_acc_RMSE, val_acc_R2 = validate(model, partition, loss_fn, args)
        te = time.time()

        # ====== Add Epoch Data ====== #
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        train_accs_RMSE.append(train_acc_RMSE)
        val_accs_RMSE.append(val_acc_RMSE)
        train_accs_R2.append(train_acc_R2)
        val_accs_R2.append(val_acc_R2)
        # ============================ #

        # # ===== Visdom visualizing ============================================================================== #
        axis.append(epoch+1)
        
        vis.line(Y=torch.cat((torch.Tensor(train_losses).view(-1,1), torch.Tensor(val_losses).view(-1,1)), -1),
                 X=torch.cat((torch.Tensor(axis).view(-1,1), torch.Tensor(axis).view(-1,1)), -1),
                 win=plot1, update='replace')
        
        vis.line(Y=torch.cat((torch.Tensor(train_accs_RMSE).view(-1,1), torch.Tensor(val_accs_RMSE).view(-1,1)), -1),
                 X=torch.cat((torch.Tensor(axis).view(-1,1), torch.Tensor(axis).view(-1,1)), -1),
                 win=plot2, update='replace')
#         # # ====================================================================================================== #
        
        print('Epoch {}, Acc_RMSE(train/val): {:2.2f}/{:2.2f}, Loss(train/val) {:2.5f}/{:2.5f}. Took {:2.2f} sec'
              .format(epoch+1, train_acc_RMSE, val_acc_RMSE, train_loss, val_loss, te-ts))
        if epoch == 50:
            test_RMSE_50, test_R2_50, Pred_data_50, True_data_50 = test(model, partition, args)
            result['test_RMSE_50'] = test_RMSE_50
            result['test_R2_50'] = test_R2_50
            result['test_pred_50'] = Pred_data_50
            result['test_true_50'] = True_data_50
        if epoch == 100:
            test_RMSE_100, test_R2_100, Pred_data_100, True_data_100 = test(model, partition, args)
            result['test_RMSE_100'] = test_RMSE_100
            result['test_R2_100'] = test_R2_100
            result['test_pred_100'] = Pred_data_100
            result['test_true_100'] = True_data_100
        if epoch == 150:
            test_RMSE_150, test_R2_150, Pred_data_150, True_data_150 = test(model, partition, args)
            result['test_RMSE_150'] = test_RMSE_150
            result['test_R2_150'] = test_R2_150
            result['test_pred_150'] = Pred_data_150
            result['test_true_150'] = True_data_150
        if epoch == 200:
            test_RMSE_200, test_R2_200, Pred_data_200, True_data_200 = test(model, partition, args)
            result['test_RMSE_200'] = test_RMSE_200
            result['test_R2_200'] = test_R2_200
            result['test_pred_200'] = Pred_data_200
            result['test_true_200'] = True_data_200
        if epoch == 300:
            test_RMSE_300, test_R2_300, Pred_data_300, True_data_300 = test(model, partition, args)
            result['test_RMSE_300'] = test_RMSE_300
            result['test_R2_300'] = test_R2_300
            result['test_pred_300'] = Pred_data_300
            result['test_true_300'] = True_data_300
        if epoch == 500:
            test_RMSE_500, test_R2_500, Pred_data_500, True_data_500 = test(model, partition, args)
            result['test_RMSE_500'] = test_RMSE_500
            result['test_R2_500'] = test_R2_500
            result['test_pred_500'] = Pred_data_500
            result['test_true_500'] = True_data_500
        if epoch == 750:
            test_RMSE_750, test_R2_750, Pred_data_750, True_data_750 = test(model, partition, args)
            result['test_RMSE_750'] = test_RMSE_750
            result['test_R2_750'] = test_R2_750
            result['test_pred_750'] = Pred_data_750
            result['test_true_750'] = True_data_750
            
    test_acc_RMSE, test_acc_R2, Pred_data, True_data = test(model, partition, args)
    
    # ======= Add Result to Dictionary ======= #
#     result = {}
    result['train_losses'] = train_losses
    result['val_losses'] = val_losses
    
    result['train_accs_RMSE'] = train_accs_RMSE
    result['train_accs_R2'] = train_accs_R2
    result['val_accs_RMSE'] = val_accs_RMSE
    result['val_accs_R2'] = val_accs_R2
#     result['train_acc'] = train_acc
#     result['val_acc'] = val_acc
    result['test_RMSE'] = test_acc_RMSE
    result['test_R2'] = test_acc_R2
    result['test_pred'] = Pred_data
    result['test_true'] = True_data
    return result

# 6. LSTM Run

In [34]:
# ====== Random Seed Initialization ====== #
seed = 666
np.random.seed(seed)
torch.manual_seed(seed)

parser = argparse.ArgumentParser()
args = parser.parse_args("")

args.device = 'cuda' if torch.cuda.is_available() else 'cpu'

# ====== Data Loading ====== #
args.batch_size = 4 # 2~8
args.UpData = UpData
args.DownData = DownData
args.x_frames = 4
args.y_frames = 1

# ====== Model Capacity ===== #
args.input_dim = len(UpData.columns)
args.hid_dim = 32
args.n_layers = 2

# ====== Regularization ======= #
args.l2 = 0.0001 
args.dropout = 0.1 
args.use_bn = False

# ====== Optimizer & Training ====== #
args.optim = 'Adam'  #SGD, RMSprop, Adam...
args.loss = 'MSELoss'#'MSELoss','L1Loss','PoissonNLLLoss','KLDivLoss','BCELoss','BCEWithLogitsLoss'
args.lr = 0.01
args.epoch = 3


# ====== Experiment Variable ====== #
name_var1 = 'x_frames'
name_var2 = 'batch_size'
name_var3 = 'optim'
name_var4 = 'lr'
name_var5 = 'epoch'
list_var1 = [4]
list_var2 = [8]
list_var3 = ['Adam']
list_var4 = [0.01]
list_var5 = [1000]


num = 0 #초기화


trainset = RiverDataset(args.UpData, args.DownData, args.x_frames, args.y_frames, '2013-01-01', '2016-07-31')
valset = RiverDataset(args.UpData, args.DownData, args.x_frames, args.y_frames, '2016-08-01', '2017-05-19')
testset = RiverDataset(args.UpData, args.DownData, args.x_frames, args.y_frames, '2016-08-01', '2017-05-19')
partition = {'train': trainset, 'val':valset, 'test':testset}


print('size of trainset :{}'.format(len(trainset)),
      'size of valset :{}'.format(len(valset)),
      'size of testset :{}'.format(len(testset)))

for var1 in list_var1:
    for var2 in list_var2:
        for var3 in list_var3:
            for var4 in list_var4:
                for var5 in list_var5:
                    num += 1
                    setattr(args, name_var1, var1)
                    setattr(args, name_var2, var2)
                    setattr(args, name_var3, var3)
                    setattr(args, name_var4, var4)
                    setattr(args, name_var5, var5)
                    print('\n exp_{}'.format(num))
                    print('loseFunc = {}, optim={}, x_frames={}, n_layers={}, batch_size={}, hid_dim={}, epoch={}, lr={}, l2={}, dropout={}, use_bn={}'
                          .format(args.loss,args.optim,args.x_frames,args.n_layers,args.batch_size,args.hid_dim,args.epoch,args.lr,args.l2,args.dropout,args.use_bn))     

                    result = experiment(partition, deepcopy(args))

            #         print('train_acc_RMSE = {:2.2f}%, val_acc_RMSE = {:2.2f}%, test_RMSE = {:2.2f}%, test_R2 = {:2.2f}%'
            #               .format(result['train_acc_RMSE'],result['val_acc_RMSE'],result['test_RMSE'],result['test_R2']))

                    vis.text('loseFunc = {}, optim={}, x_frames={}, n_layers={}, batch_size={}, hid_dim={}, epoch={}, lr={}, l2={}, dropout={}, use_bn={}'
                             .format(args.loss,args.optim,args.x_frames,args.n_layers,args.batch_size,args.hid_dim,args.epoch,args.lr,args.l2,args.dropout,args.use_bn),
                             opts=dict(title='exp_{}_text'.format(num)))
                    # 만든 모델의 test 데이터 예측 시각화

                    predict = torch.Tensor(result['test_pred']).view(-1,1)
                    truth = torch.Tensor(result['test_true']).view(-1,1)
                    axis = torch.Tensor(range(len(result['test_pred']))).view(-1,1)
                    Y_axis = torch.cat((predict, truth), -1)
                    X_axis = torch.cat((axis, axis), -1)
                    vis.line(Y = Y_axis, X = X_axis, opts=dict(title='Result_exp_{}_RMSE[{:2.3f}]_R2[{:2.3f}]'
                                                               .format(num,result['test_RMSE'],result['test_R2']),
                                                               legend=['predict','true'],
                                                               showlegend=True,
                                                               layoutopts = {'plotly': {'legend': {'x':0, 'y':0}}}))
                    
                    if args.epoch >= 50:
                        predict = torch.Tensor(result['test_pred_50']).view(-1,1)
                        truth = torch.Tensor(result['test_true_50']).view(-1,1)
                        axis = torch.Tensor(range(len(result['test_pred_50']))).view(-1,1)
                        Y_axis = torch.cat((predict, truth), -1)
                        X_axis = torch.cat((axis, axis), -1)
                        vis.line(Y = Y_axis, X = X_axis, opts=dict(title='Result_exp_{}_epoch[50]_RMSE[{:2.3f}]_R2[{:2.3f}]'
                                                                   .format(num,result['test_RMSE_50'],result['test_R2_50']),
                                                                   legend=['predict','true'],
                                                                   showlegend=True,
                                                                   layoutopts = {'plotly': {'legend': {'x':0, 'y':0}}}))


                    if args.epoch >= 100:
                        predict = torch.Tensor(result['test_pred_100']).view(-1,1)
                        truth = torch.Tensor(result['test_true_100']).view(-1,1)
                        axis = torch.Tensor(range(len(result['test_pred_100']))).view(-1,1)
                        Y_axis = torch.cat((predict, truth), -1)
                        X_axis = torch.cat((axis, axis), -1)
                        vis.line(Y = Y_axis, X = X_axis, opts=dict(title='Result_exp_{}_epoch[100]_RMSE[{:2.3f}]_R2[{:2.3f}]'
                                                                   .format(num,result['test_RMSE_100'],result['test_R2_100']),
                                                                   legend=['predict','true'],
                                                                   showlegend=True,
                                                                   layoutopts = {'plotly': {'legend': {'x':0, 'y':0}}}))

                    if args.epoch >= 150:
                        predict = torch.Tensor(result['test_pred_150']).view(-1,1)
                        truth = torch.Tensor(result['test_true_150']).view(-1,1)
                        axis = torch.Tensor(range(len(result['test_pred_150']))).view(-1,1)
                        Y_axis = torch.cat((predict, truth), -1)
                        X_axis = torch.cat((axis, axis), -1)
                        vis.line(Y = Y_axis, X = X_axis, opts=dict(title='Result_exp_{}_epoch[150]_RMSE[{:2.3f}]_R2[{:2.3f}]'
                                                                   .format(num,result['test_RMSE_150'],result['test_R2_150']),
                                                                   legend=['predict','true'],
                                                                   showlegend=True,
                                                                   layoutopts = {'plotly': {'legend': {'x':0, 'y':0}}}))
                    
                    if args.epoch >= 200:
                        predict = torch.Tensor(result['test_pred_200']).view(-1,1)
                        truth = torch.Tensor(result['test_true_200']).view(-1,1)
                        axis = torch.Tensor(range(len(result['test_pred_200']))).view(-1,1)
                        Y_axis = torch.cat((predict, truth), -1)
                        X_axis = torch.cat((axis, axis), -1)
                        vis.line(Y = Y_axis, X = X_axis, opts=dict(title='Result_exp_{}_epoch[200]_RMSE[{:2.3f}]_R2[{:2.3f}]'
                                                                   .format(num,result['test_RMSE_200'],result['test_R2_200']),
                                                                   legend=['predict','true'],
                                                                   showlegend=True,
                                                                   layoutopts = {'plotly': {'legend': {'x':0, 'y':0}}}))
                        
                    if args.epoch >= 300:
                        predict = torch.Tensor(result['test_pred_300']).view(-1,1)
                        truth = torch.Tensor(result['test_true_300']).view(-1,1)
                        axis = torch.Tensor(range(len(result['test_pred_300']))).view(-1,1)
                        Y_axis = torch.cat((predict, truth), -1)
                        X_axis = torch.cat((axis, axis), -1)
                        vis.line(Y = Y_axis, X = X_axis, opts=dict(title='Result_exp_{}_epoch[300]_RMSE[{:2.3f}]_R2[{:2.3f}]'
                                                                   .format(num,result['test_RMSE_300'],result['test_R2_300']),
                                                                   legend=['predict','true'],
                                                                   showlegend=True,
                                                                   layoutopts = {'plotly': {'legend': {'x':0, 'y':0}}}))
                        
                    if args.epoch >= 500:
                        predict = torch.Tensor(result['test_pred_500']).view(-1,1)
                        truth = torch.Tensor(result['test_true_500']).view(-1,1)
                        axis = torch.Tensor(range(len(result['test_pred_500']))).view(-1,1)
                        Y_axis = torch.cat((predict, truth), -1)
                        X_axis = torch.cat((axis, axis), -1)
                        vis.line(Y = Y_axis, X = X_axis, opts=dict(title='Result_exp_{}_epoch[500]_RMSE[{:2.3f}]_R2[{:2.3f}]'
                                                                   .format(num,result['test_RMSE_500'],result['test_R2_500']),
                                                                   legend=['predict','true'],
                                                                   showlegend=True,
                                                                   layoutopts = {'plotly': {'legend': {'x':0, 'y':0}}}))
                        
                    if args.epoch >= 750:
                        predict = torch.Tensor(result['test_pred_750']).view(-1,1)
                        truth = torch.Tensor(result['test_true_750']).view(-1,1)
                        axis = torch.Tensor(range(len(result['test_pred_750']))).view(-1,1)
                        Y_axis = torch.cat((predict, truth), -1)
                        X_axis = torch.cat((axis, axis), -1)
                        vis.line(Y = Y_axis, X = X_axis, opts=dict(title='Result_exp_{}_epoch[750]_RMSE[{:2.3f}]_R2[{:2.3f}]'
                                                                   .format(num,result['test_RMSE_750'],result['test_R2_750']),
                                                                   legend=['predict','true'],
                                                                   showlegend=True,
                                                                   layoutopts = {'plotly': {'legend': {'x':0, 'y':0}}}))





print('All done')

size of trainset :181 size of valset :38 size of testset :38

 exp_1
loseFunc = MSELoss, optim=Adam, x_frames=4, n_layers=2, batch_size=8, hid_dim=32, epoch=1000, lr=0.01, l2=0.0001, dropout=0.1, use_bn=False
Epoch 0, Acc_RMSE(train/val): 0.04/0.03, Loss(train/val) 0.06323/0.06664. Took 0.22 sec
Epoch 1, Acc_RMSE(train/val): 0.03/0.03, Loss(train/val) 0.03704/0.05777. Took 0.23 sec
Epoch 2, Acc_RMSE(train/val): 0.03/0.03, Loss(train/val) 0.05026/0.07486. Took 0.23 sec
Epoch 3, Acc_RMSE(train/val): 0.03/0.03, Loss(train/val) 0.03912/0.07363. Took 0.23 sec
Epoch 4, Acc_RMSE(train/val): 0.03/0.03, Loss(train/val) 0.03723/0.06532. Took 0.23 sec
Epoch 5, Acc_RMSE(train/val): 0.03/0.03, Loss(train/val) 0.03777/0.05850. Took 0.22 sec
Epoch 6, Acc_RMSE(train/val): 0.03/0.03, Loss(train/val) 0.03918/0.06031. Took 0.22 sec
Epoch 7, Acc_RMSE(train/val): 0.03/0.03, Loss(train/val) 0.03506/0.04929. Took 0.21 sec
Epoch 8, Acc_RMSE(train/val): 0.03/0.03, Loss(train/val) 0.03475/0.05373. Took 0.24 sec

Epoch 90, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.01350/0.02151. Took 0.24 sec
Epoch 91, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.01287/0.02073. Took 0.22 sec
Epoch 92, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.01315/0.02077. Took 0.24 sec
Epoch 93, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.01171/0.02061. Took 0.22 sec
Epoch 94, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.01258/0.01978. Took 0.23 sec
Epoch 95, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.01222/0.02006. Took 0.22 sec
Epoch 96, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.01228/0.01985. Took 0.24 sec
Epoch 97, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.01215/0.01909. Took 0.22 sec
Epoch 98, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.01252/0.01986. Took 0.23 sec
Epoch 99, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.01227/0.01897. Took 0.22 sec
Epoch 100, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.01271/0.01946. Took 0.24 sec
Epoch 101, Acc_RMSE(

Epoch 182, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.01385/0.02098. Took 0.23 sec
Epoch 183, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.01165/0.01891. Took 0.21 sec
Epoch 184, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.01135/0.01980. Took 0.22 sec
Epoch 185, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.01146/0.01948. Took 0.22 sec
Epoch 186, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.01108/0.01847. Took 0.22 sec
Epoch 187, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.01094/0.02091. Took 0.22 sec
Epoch 188, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.01235/0.02053. Took 0.24 sec
Epoch 189, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.01354/0.02126. Took 0.21 sec
Epoch 190, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.01289/0.02095. Took 0.22 sec
Epoch 191, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.01223/0.01931. Took 0.22 sec
Epoch 192, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.01227/0.02026. Took 0.23 sec
Epoch 193,

Epoch 274, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.01671/0.02047. Took 0.22 sec
Epoch 275, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.01256/0.01939. Took 0.22 sec
Epoch 276, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.01139/0.01812. Took 0.23 sec
Epoch 277, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.01089/0.01873. Took 0.22 sec
Epoch 278, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.01144/0.01736. Took 0.22 sec
Epoch 279, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.01018/0.01741. Took 0.22 sec
Epoch 280, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.01038/0.02294. Took 0.23 sec
Epoch 281, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.01458/0.02044. Took 0.22 sec
Epoch 282, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.01070/0.01917. Took 0.23 sec
Epoch 283, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.01363/0.01929. Took 0.22 sec
Epoch 284, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.01129/0.02035. Took 0.22 sec
Epoch 285,

Epoch 366, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.01173/0.02179. Took 0.23 sec
Epoch 367, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.00956/0.02075. Took 0.22 sec
Epoch 368, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.00967/0.01857. Took 0.22 sec
Epoch 369, Acc_RMSE(train/val): 0.01/0.02, Loss(train/val) 0.01018/0.02066. Took 0.22 sec
Epoch 370, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.00983/0.02264. Took 0.22 sec
Epoch 371, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.01040/0.02270. Took 0.23 sec
Epoch 372, Acc_RMSE(train/val): 0.01/0.02, Loss(train/val) 0.00881/0.02290. Took 0.22 sec
Epoch 373, Acc_RMSE(train/val): 0.01/0.02, Loss(train/val) 0.00892/0.02213. Took 0.22 sec
Epoch 374, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.01194/0.02630. Took 0.22 sec
Epoch 375, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.01220/0.02424. Took 0.22 sec
Epoch 376, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.01332/0.02010. Took 0.22 sec
Epoch 377,

...

Epoch 550, Acc_RMSE(train/val): 0.01/0.02, Loss(train/val) 0.00732/0.02330. Took 0.22 sec
Epoch 551, Acc_RMSE(train/val): 0.01/0.02, Loss(train/val) 0.00876/0.02583. Took 0.22 sec
Epoch 552, Acc_RMSE(train/val): 0.01/0.02, Loss(train/val) 0.00892/0.03105. Took 0.22 sec
Epoch 553, Acc_RMSE(train/val): 0.01/0.02, Loss(train/val) 0.00770/0.02422. Took 0.22 sec
Epoch 554, Acc_RMSE(train/val): 0.01/0.02, Loss(train/val) 0.00717/0.02770. Took 0.22 sec
Epoch 555, Acc_RMSE(train/val): 0.01/0.02, Loss(train/val) 0.00734/0.02431. Took 0.23 sec
Epoch 556, Acc_RMSE(train/val): 0.01/0.02, Loss(train/val) 0.00730/0.02760. Took 0.22 sec
Epoch 557, Acc_RMSE(train/val): 0.01/0.02, Loss(train/val) 0.00759/0.02847. Took 0.21 sec
Epoch 558, Acc_RMSE(train/val): 0.01/0.02, Loss(train/val) 0.00842/0.02197. Took 0.22 sec
Epoch 559, Acc_RMSE(train/val): 0.01/0.02, Loss(train/val) 0.00749/0.02647. Took 0.22 sec
Epoch 560, Acc_RMSE(train/val): 0.01/0.02, Loss(train/val) 0.00703/0.02214. Took 0.22 sec
Epoch 561,

Epoch 642, Acc_RMSE(train/val): 0.01/0.02, Loss(train/val) 0.00780/0.03028. Took 0.22 sec
Epoch 643, Acc_RMSE(train/val): 0.01/0.02, Loss(train/val) 0.00891/0.02867. Took 0.22 sec
Epoch 644, Acc_RMSE(train/val): 0.01/0.02, Loss(train/val) 0.00743/0.02638. Took 0.22 sec
Epoch 645, Acc_RMSE(train/val): 0.01/0.02, Loss(train/val) 0.00680/0.03054. Took 0.22 sec
Epoch 646, Acc_RMSE(train/val): 0.01/0.02, Loss(train/val) 0.00739/0.02716. Took 0.22 sec
Epoch 647, Acc_RMSE(train/val): 0.01/0.02, Loss(train/val) 0.00697/0.02696. Took 0.22 sec
Epoch 648, Acc_RMSE(train/val): 0.01/0.02, Loss(train/val) 0.00839/0.02637. Took 0.22 sec
Epoch 649, Acc_RMSE(train/val): 0.01/0.02, Loss(train/val) 0.00775/0.02519. Took 0.22 sec
Epoch 650, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.01090/0.02864. Took 0.22 sec
Epoch 651, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.01015/0.02747. Took 0.23 sec
Epoch 652, Acc_RMSE(train/val): 0.01/0.02, Loss(train/val) 0.00841/0.02838. Took 0.22 sec
Epoch 653,

Epoch 734, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.00893/0.02728. Took 0.22 sec
Epoch 735, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.01018/0.02530. Took 0.22 sec
Epoch 736, Acc_RMSE(train/val): 0.01/0.02, Loss(train/val) 0.00776/0.02646. Took 0.23 sec
Epoch 737, Acc_RMSE(train/val): 0.01/0.02, Loss(train/val) 0.00764/0.02746. Took 0.22 sec
Epoch 738, Acc_RMSE(train/val): 0.01/0.02, Loss(train/val) 0.00755/0.02471. Took 0.22 sec
Epoch 739, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.00792/0.02553. Took 0.22 sec
Epoch 740, Acc_RMSE(train/val): 0.01/0.02, Loss(train/val) 0.00795/0.02553. Took 0.22 sec
Epoch 741, Acc_RMSE(train/val): 0.01/0.02, Loss(train/val) 0.00720/0.02746. Took 0.23 sec
Epoch 742, Acc_RMSE(train/val): 0.01/0.02, Loss(train/val) 0.00589/0.02709. Took 0.22 sec
Epoch 743, Acc_RMSE(train/val): 0.01/0.02, Loss(train/val) 0.00687/0.02896. Took 0.22 sec
Epoch 744, Acc_RMSE(train/val): 0.01/0.02, Loss(train/val) 0.00759/0.02760. Took 0.22 sec
Epoch 745,

Epoch 826, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.00919/0.02519. Took 0.22 sec
Epoch 827, Acc_RMSE(train/val): 0.01/0.02, Loss(train/val) 0.00764/0.02514. Took 0.23 sec
Epoch 828, Acc_RMSE(train/val): 0.01/0.02, Loss(train/val) 0.00699/0.02766. Took 0.22 sec
Epoch 829, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.00823/0.03024. Took 0.22 sec
Epoch 830, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.00848/0.02881. Took 0.22 sec
Epoch 831, Acc_RMSE(train/val): 0.01/0.02, Loss(train/val) 0.00863/0.02410. Took 0.23 sec
Epoch 832, Acc_RMSE(train/val): 0.01/0.02, Loss(train/val) 0.00692/0.02618. Took 0.22 sec
Epoch 833, Acc_RMSE(train/val): 0.01/0.02, Loss(train/val) 0.00623/0.02607. Took 0.22 sec
Epoch 834, Acc_RMSE(train/val): 0.01/0.02, Loss(train/val) 0.00717/0.02831. Took 0.22 sec
Epoch 835, Acc_RMSE(train/val): 0.01/0.02, Loss(train/val) 0.00680/0.02619. Took 0.23 sec
Epoch 836, Acc_RMSE(train/val): 0.01/0.02, Loss(train/val) 0.00747/0.02498. Took 0.22 sec
Epoch 837,

Epoch 918, Acc_RMSE(train/val): 0.01/0.02, Loss(train/val) 0.00685/0.02651. Took 0.22 sec
Epoch 919, Acc_RMSE(train/val): 0.01/0.02, Loss(train/val) 0.00725/0.02767. Took 0.22 sec
Epoch 920, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.00992/0.02451. Took 0.22 sec
Epoch 921, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.00896/0.02304. Took 0.22 sec
Epoch 922, Acc_RMSE(train/val): 0.01/0.02, Loss(train/val) 0.00668/0.02630. Took 0.22 sec
Epoch 923, Acc_RMSE(train/val): 0.01/0.02, Loss(train/val) 0.00593/0.02554. Took 0.22 sec
Epoch 924, Acc_RMSE(train/val): 0.01/0.02, Loss(train/val) 0.00637/0.02729. Took 0.22 sec
Epoch 925, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.00917/0.02883. Took 0.22 sec
Epoch 926, Acc_RMSE(train/val): 0.01/0.02, Loss(train/val) 0.00780/0.02619. Took 0.23 sec
Epoch 927, Acc_RMSE(train/val): 0.01/0.02, Loss(train/val) 0.00713/0.02784. Took 0.22 sec
Epoch 928, Acc_RMSE(train/val): 0.01/0.02, Loss(train/val) 0.00681/0.02780. Took 0.23 sec
Epoch 929,

In [ ]:
lose = ['MSELoss','L1Loss','PoissonNLLLoss','KLDivLoss','BCELoss','BCEWithLogitsLoss']
optim = ['SGD', 'RMSprop', 'Adam']
dropout = ['']